# Run temperature attributions and save out

1. Run one scenario emissions driven. Save out the following forcing categories:

- All forcing
- All minus GHGs
- All minus aerosols
- All minus other anthropogenic
- All minus natural

2. re-run forcing-driven scenarios with the above.

In [ ]:
import os

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import scipy.stats
import xarray as xr

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
output_ensemble_size=841

In [ ]:
scenarios = ["all"]

In [ ]:
df_solar = pd.read_csv(
    "../data/forcing/solar_erf_timebounds.csv", index_col="year"
)
df_volcanic = pd.read_csv(
    "../data/forcing/volcanic_ERF_1750-2101_timebounds.csv",
    index_col="timebounds",
)

# Normally, we adjust the efficacy of volcanic forcing, but here we calculate temperatures from forcing, so adjust the forcing directly.
volcanic_forcing = df_volcanic["erf"].loc[1750:2023].values * 0.6
solar_forcing = df_solar["erf"].loc[1750:2023].values

trend_shape = np.ones(274)
trend_shape[:271] = np.linspace(0, 1, 271)

In [ ]:
df_methane_141 = pd.read_csv(
    "../data/calibration/v1.4.1/CH4_lifetime.csv",
    index_col=0,
)
df_configs_141 = pd.read_csv(
    "../data/calibration/v1.4.1/calibrated_constrained_parameters.csv",
    index_col=0,
)
df_landuse_141 = pd.read_csv(
    "../data/calibration/v1.4.1/landuse_scale_factor.csv",
    index_col=0,
)
df_lapsi_141 = pd.read_csv(
    "../data/calibration/v1.4.1/lapsi_scale_factor.csv",
    index_col=0,
)

In [ ]:
configs = df_configs_141.index

## First, emissions-driven run

In [ ]:
f = FAIR(ch4_method="Thornhill2021")
f.define_time(1750, 2023, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
species, properties = read_properties()
species.remove("Halon-1202")
species.remove("NOx aviation")
species.remove("Contrails")
f.define_species(species, properties)
f.allocate()

### Get emissions into fair

copy across the baseline emissions from ssp245 into all scenarios (baseline scenario doesn't matter as they are the same up to 2022)

In [ ]:
da_emissions_141 = xr.load_dataarray(
    "../data/emissions/v1.4.1/ssps_harmonized_1750-2499.nc"
)
da = da_emissions_141.loc[
    dict(config="unspecified", scenario="ssp245", timepoints=np.arange(1750.5, 2023))
].drop_vars(("scenario", "config"))

for config in configs:
    for scenario in scenarios:
        f.emissions.loc[dict(scenario=scenario, config=config)] = da

In [ ]:
f.emissions

### Fill in all the configs

In [ ]:
# solar and volcanic forcing
fill(
    f.forcing,
    volcanic_forcing[:, None, None] * df_configs_141["fscale_Volcanic"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    solar_forcing[:, None, None] * df_configs_141["fscale_solar_amplitude"].values.squeeze()
    + trend_shape[:, None, None] * df_configs_141["fscale_solar_trend"].values.squeeze(),
    specie="Solar",
)
# climate response
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs_141.loc[:, "clim_c1":"clim_c3"].values,
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs_141.loc[:, "clim_kappa1":"clim_kappa3"].values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    f.climate_configs["deep_ocean_efficacy"],
    df_configs_141["clim_epsilon"].values.squeeze(),
)
fill(
    f.climate_configs["gamma_autocorrelation"],
    df_configs_141["clim_gamma"].values.squeeze(),
)
fill(f.climate_configs["sigma_eta"], df_configs_141["clim_sigma_eta"].values.squeeze())
fill(f.climate_configs["sigma_xi"], df_configs_141["clim_sigma_xi"].values.squeeze())
fill(f.climate_configs["seed"], df_configs_141["seed"])
fill(f.climate_configs["stochastic_run"], False)
fill(f.climate_configs["use_seed"], False)
fill(f.climate_configs["forcing_4co2"], df_configs_141["clim_F_4xCO2"])

# species level
f.fill_species_configs()

# carbon cycle
fill(f.species_configs["iirf_0"], df_configs_141["cc_r0"].values.squeeze(), specie="CO2")
fill(
    f.species_configs["iirf_airborne"],
    df_configs_141["cc_rA"].values.squeeze(),
    specie="CO2",
)
fill(
    f.species_configs["iirf_uptake"], df_configs_141["cc_rU"].values.squeeze(), specie="CO2"
)
fill(
    f.species_configs["iirf_temperature"],
    df_configs_141["cc_rT"].values.squeeze(),
    specie="CO2",
)

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs_141["aci_beta"].values.squeeze())
fill(
    f.species_configs["aci_shape"],
    df_configs_141["aci_shape_so2"].values.squeeze(),
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"],
    df_configs_141["aci_shape_bc"].values.squeeze(),
    specie="BC",
)
fill(
    f.species_configs["aci_shape"],
    df_configs_141["aci_shape_oc"].values.squeeze(),
    specie="OC",
)

# methane lifetime baseline and sensitivity
fill(
    f.species_configs["unperturbed_lifetime"],
    df_methane_141.loc["historical_best", "base"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane_141.loc["historical_best", "CH4"],
    specie="CH4",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane_141.loc["historical_best", "N2O"],
    specie="N2O",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane_141.loc["historical_best", "VOC"],
    specie="VOC",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane_141.loc["historical_best", "NOx"],
    specie="NOx",
)
fill(
    f.species_configs["ch4_lifetime_chemical_sensitivity"],
    df_methane_141.loc["historical_best", "HC"],
    specie="Equivalent effective stratospheric chlorine",
)
fill(
    f.species_configs["lifetime_temperature_sensitivity"],
    df_methane_141.loc["historical_best", "temp"],
)

# correct land use  and LAPSI scale factor terms
fill(
    f.species_configs["land_use_cumulative_emissions_to_forcing"],
    df_landuse_141.loc["historical_best", "CO2_AFOLU"],
    specie="CO2 AFOLU",
)
fill(
    f.species_configs["lapsi_radiative_efficiency"],
    df_lapsi_141.loc["historical_best", "BC"],
    specie="BC",
)

# emissions adjustments for N2O and CH4 (we don't want to make these defaults as people
# might wanna run pulse expts with these gases)
fill(f.species_configs["baseline_emissions"], 38.246272, specie="CH4")
fill(f.species_configs["baseline_emissions"], 0.92661989, specie="N2O")
fill(f.species_configs["baseline_emissions"], 19.41683292, specie="NOx")
fill(f.species_configs["baseline_emissions"], 2.293964929, specie="Sulfur")
fill(f.species_configs["baseline_emissions"], 348.4549732, specie="CO")
fill(f.species_configs["baseline_emissions"], 60.62284009, specie="VOC")
fill(f.species_configs["baseline_emissions"], 2.096765609, specie="BC")
fill(f.species_configs["baseline_emissions"], 15.44571911, specie="OC")
fill(f.species_configs["baseline_emissions"], 6.656462698, specie="NH3")
fill(f.species_configs["baseline_emissions"], 0.92661989, specie="N2O")
fill(f.species_configs["baseline_emissions"], 0.02129917, specie="CCl4")
fill(f.species_configs["baseline_emissions"], 202.7251231, specie="CHCl3")
fill(f.species_configs["baseline_emissions"], 211.0095537, specie="CH2Cl2")
fill(f.species_configs["baseline_emissions"], 4544.519056, specie="CH3Cl")
fill(f.species_configs["baseline_emissions"], 111.4920237, specie="CH3Br")
fill(f.species_configs["baseline_emissions"], 0.008146006, specie="Halon-1211")
fill(f.species_configs["baseline_emissions"], 0.000010554155, specie="SO2F2")
fill(f.species_configs["baseline_emissions"], 0, specie="CF4")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
    "Equivalent effective stratospheric chlorine",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs_141[f"ari_{specie}"],
        specie=specie,
    )

# forcing scaling
for specie in [
    "CO2",
    "CH4",
    "N2O",
    "Stratospheric water vapour",
    "Light absorbing particles on snow and ice",
    "Land use",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs_141[f"fscale_{specie}"].values.squeeze(),
        specie=specie,
    )

for specie in [
    "CFC-11",
    "CFC-12",
    "CFC-113",
    "CFC-114",
    "CFC-115",
    "HCFC-22",
    "HCFC-141b",
    "HCFC-142b",
    "CCl4",
    "CHCl3",
    "CH2Cl2",
    "CH3Cl",
    "CH3CCl3",
    "CH3Br",
    "Halon-1211",
    "Halon-1301",
    "Halon-2402",
    "CF4",
    "C2F6",
    "C3F8",
    "c-C4F8",
    "C4F10",
    "C5F12",
    "C6F14",
    "C7F16",
    "C8F18",
    "NF3",
    "SF6",
    "SO2F2",
    "HFC-125",
    "HFC-134a",
    "HFC-143a",
    "HFC-152a",
    "HFC-227ea",
    "HFC-23",
    "HFC-236fa",
    "HFC-245fa",
    "HFC-32",
    "HFC-365mfc",
    "HFC-4310mee",
]:
    fill(
        f.species_configs["forcing_scale"],
        df_configs_141["fscale_minorGHG"].values.squeeze(),
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "Equivalent effective stratospheric chlorine",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs_141[f"o3_{specie}"],
        specie=specie,
    )

# # tune down volcanic efficacy
# fill(f.species_configs["forcing_efficacy"], 0.6, specie="Volcanic")


# initial condition of CO2 concentration (but not baseline for forcing calculations)
fill(
    f.species_configs["baseline_concentration"],
    df_configs_141["cc_co2_concentration_1750"].values.squeeze(),
    specie="CO2",
)

# initial conditions
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)

In [ ]:
f.run()

## Now isolate out forcing cats

In [ ]:
f.species

In [ ]:
ghgs = [
 'CO2',
 'CH4',
 'N2O',
 'CFC-11',
 'CFC-12',
 'CFC-113',
 'CFC-114',
 'CFC-115',
 'HCFC-22',
 'HCFC-141b',
 'HCFC-142b',
 'CCl4',
 'CHCl3',
 'CH2Cl2',
 'CH3Cl',
 'CH3CCl3',
 'CH3Br',
 'Halon-1211',
 'Halon-1301',
 'Halon-2402',
 'CF4',
 'C2F6',
 'C3F8',
 'c-C4F8',
 'C4F10',
 'C5F12',
 'C6F14',
 'C7F16',
 'C8F18',
 'NF3',
 'SF6',
 'SO2F2',
 'HFC-125',
 'HFC-134a',
 'HFC-143a',
 'HFC-152a',
 'HFC-227ea',
 'HFC-23',
 'HFC-236fa',
 'HFC-245fa',
 'HFC-32',
 'HFC-365mfc',
 'HFC-4310mee',
]

In [ ]:
aerosols = [
 'Aerosol-radiation interactions',
 'Aerosol-cloud interactions',
]

In [ ]:
natural = [
 'Solar',
 'Volcanic',
]

In [ ]:
other = [
 'Ozone',
 'Light absorbing particles on snow and ice',
 'Stratospheric water vapour',
 'Land use',
]

In [ ]:
pl.plot(f.forcing.sel(specie=ghgs).sum(dim='specie').sel(scenario='all'));

In [ ]:
pl.plot(f.forcing.sel(specie=aerosols).sum(dim='specie').sel(scenario='all'));

In [ ]:
pl.plot(f.forcing.sel(specie=other).sum(dim='specie').sel(scenario='all'));

In [ ]:
pl.plot(f.forcing.sel(specie=natural).sum(dim='specie').sel(scenario='all'));

### Check linearity

note y-axis value very small - minimal floating point differences are fine

In [ ]:
pl.plot(
    f.forcing_sum.sel(scenario='all') - (
        f.forcing.sel(specie=ghgs).sum(dim='specie').sel(scenario='all') +
        f.forcing.sel(specie=aerosols).sum(dim='specie').sel(scenario='all') +
        f.forcing.sel(specie=other).sum(dim='specie').sel(scenario='all') + 
        f.forcing.sel(specie=natural).sum(dim='specie').sel(scenario='all')
    )
);

## Constrained ensemble statistics

Compare the constrained ensemble to the IPCC assessed ranges. ECS is asymmetric so we fit a skew-normal to the percentiles.

In [ ]:
def opt(x, q05_desired, q50_desired, q95_desired):
    """Fit a skew-normal distribution to 5, 50, 95 percentiles.
    
    x is (a, loc, scale) in that order."""
    q05, q50, q95 = scipy.stats.skewnorm.ppf(
        (0.05, 0.50, 0.95), x[0], loc=x[1], scale=x[2]
    )
    return (q05 - q05_desired, q50 - q50_desired, q95 - q95_desired)

In [ ]:
# conversion from one standard deviation to 90% range
NINETY_TO_ONESIGMA = scipy.stats.norm.ppf(0.95)
NINETY_TO_ONESIGMA

In [ ]:
ecs_params = scipy.optimize.root(opt, [1, 1, 1], args=(2, 3, 5)).x
ecs_params

In [ ]:
aer_dist = scipy.stats.norm(loc=-1.3, scale=0.7/NINETY_TO_ONESIGMA)

In [ ]:
ecs_samp = ((df_configs_141['clim_F_4xCO2']/2)/df_configs_141['clim_kappa1']).values
ecs_dist = scipy.stats.skewnorm(a=ecs_params[0], loc=ecs_params[1], scale=ecs_params[2])

In [ ]:
aer_samp = f.forcing.sel(specie=aerosols, timebounds=np.arange(2005, 2015)).sum(dim='specie').sel(scenario='all').mean(dim='timebounds')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(12,4))

ax[0].hist(aer_samp, bins=np.arange(-2.5, 0.1, 0.1), density=True)
ax[0].plot(np.linspace(-3, 0.5, 101), aer_dist.pdf(np.linspace(-3, 0.5, 101)), color='k', lw=2)
ax[0].set_xlim(-3, 0.5)
ax[0].set_title('Aerosol forcing')
ax[0].set_xlabel('W m$^{-2}$')
ax[0].set_ylabel('Density')

ax[1].hist(ecs_samp, bins=np.arange(0, 8.1, 0.2), density=True, label='Constrained ensemble')
ax[1].plot(np.linspace(0, 8, 101), ecs_dist.pdf(np.linspace(0, 8, 101)), color='k', lw=2, label='IPCC AR6')
ax[1].set_xlim(0, 8)
ax[1].legend(fontsize=10, frameon=False)
ax[1].set_title('ECS')
ax[1].set_xlabel('ECS, °C')
ax[1].set_ylabel('Density')

ax[2].scatter(ecs_samp, aer_samp)
ax[2].set_xlim(1,8)
ax[2].set_ylim(-2.6, 0.1)
ax[2].set_title('Correlation')
ax[2].set_xlabel('ECS, °C')
ax[2].set_ylabel('Aerosol forcing, W m$^{-2}$')

fig.tight_layout()
os.makedirs('../plots/', exist_ok=True)
pl.savefig('../plots/distributions.png')

In [ ]:
scenarios = ["all", "no_ghgs", "no_aerosols", "no_other", "no_natural"]

In [ ]:
ff = FAIR()
ff.define_time(1750, 2023, 1)
ff.define_scenarios(scenarios)
ff.define_configs(configs)

species = ["bulk"]
properties = {
    "bulk": {
        "type": "unspecified",
        "input_mode": "forcing",
        "greenhouse_gas": False,
        "aerosol_chemistry_from_emissions": False,
        "aerosol_chemistry_from_concentration": False,
    }
}

ff.define_species(species, properties)
ff.allocate()

### Create forcing-driven time series

In [ ]:
fill(
    ff.forcing,
    f.forcing_sum.sel(scenario='all') - f.forcing.sel(specie=ghgs).sum(dim='specie').sel(scenario='all'),
    specie="bulk",
    scenario="no_ghgs"
)

In [ ]:
fill(
    ff.forcing,
    f.forcing_sum.sel(scenario='all') - f.forcing.sel(specie=aerosols).sum(dim='specie').sel(scenario='all'),
    specie="bulk",
    scenario="no_aerosols"
)

In [ ]:
fill(
    ff.forcing,
    f.forcing_sum.sel(scenario='all') - f.forcing.sel(specie=other).sum(dim='specie').sel(scenario='all'),
    specie="bulk",
    scenario="no_other"
)

In [ ]:
fill(
    ff.forcing,
    f.forcing_sum.sel(scenario='all') - f.forcing.sel(specie=natural).sum(dim='specie').sel(scenario='all'),
    specie="bulk",
    scenario="no_natural"
)

In [ ]:
fill(
    ff.forcing,
    f.forcing_sum.sel(scenario='all'),
    specie="bulk",
    scenario="all"
)

### fill and initialise configs

In [ ]:
# climate response
fill(
    ff.climate_configs["ocean_heat_capacity"],
    df_configs_141.loc[:, "clim_c1":"clim_c3"].values,
)
fill(
    ff.climate_configs["ocean_heat_transfer"],
    df_configs_141.loc[:, "clim_kappa1":"clim_kappa3"].values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    ff.climate_configs["deep_ocean_efficacy"],
    df_configs_141["clim_epsilon"].values.squeeze(),
)
fill(
    ff.climate_configs["gamma_autocorrelation"],
    df_configs_141["clim_gamma"].values.squeeze(),
)
fill(ff.climate_configs["sigma_eta"], df_configs_141["clim_sigma_eta"].values.squeeze())
fill(ff.climate_configs["sigma_xi"], df_configs_141["clim_sigma_xi"].values.squeeze())
fill(ff.climate_configs["seed"], df_configs_141["seed"])
fill(ff.climate_configs["stochastic_run"], False)
fill(ff.climate_configs["use_seed"], False)
fill(ff.climate_configs["forcing_4co2"], df_configs_141["clim_F_4xCO2"])

# initial conditions
initialise(ff.forcing, 0)
initialise(ff.temperature, 0)

In [ ]:
ff.run()

In [ ]:
base = np.arange(1850, 1901)
temp_ghgs = (
    (
        ff.temperature.sel(scenario="all", layer=0) - ff.temperature.sel(scenario="all", layer=0, timebounds=base).mean(dim='timebounds')
    ) - (
        ff.temperature.sel(scenario="no_ghgs", layer=0) - ff.temperature.sel(scenario="no_ghgs", layer=0, timebounds=base).mean(dim='timebounds')
    )
)
temp_aerosols = (
    (
        ff.temperature.sel(scenario="all", layer=0) - ff.temperature.sel(scenario="all", layer=0, timebounds=base).mean(dim='timebounds')
    ) - (
        ff.temperature.sel(scenario="no_aerosols", layer=0) - ff.temperature.sel(scenario="no_aerosols", layer=0, timebounds=base).mean(dim='timebounds')
    )
)
temp_natural = (
    (
        ff.temperature.sel(scenario="all", layer=0) - ff.temperature.sel(scenario="all", layer=0, timebounds=base).mean(dim='timebounds')
    ) - (
        ff.temperature.sel(scenario="no_natural", layer=0) - ff.temperature.sel(scenario="no_natural", layer=0, timebounds=base).mean(dim='timebounds')
    )
)
temp_other = (
    (
        ff.temperature.sel(scenario="all", layer=0) - ff.temperature.sel(scenario="all", layer=0, timebounds=base).mean(dim='timebounds')
    ) - (
        ff.temperature.sel(scenario="no_other", layer=0) - ff.temperature.sel(scenario="no_other", layer=0, timebounds=base).mean(dim='timebounds')
    )
)
temp_all = ff.temperature.sel(scenario="all", layer=0) - ff.temperature.sel(scenario="all", layer=0, timebounds=base).mean(dim='timebounds')

In [ ]:
df_obs = pd.read_csv('../data/observations/IGCC_GMST_1850-2022.csv', index_col=0)
df_obs

## Gillett et al. (2021) style plot

fig 1 b in https://www.nature.com/articles/s41558-020-00965-9

Note the ranges coming out of fair are much more constrained than CMIP6 models in Gillett et al., since the historical climate record and emergent climate metrics are constrained on observations and AR6 assessment (by the fair-calibrate ensemble).

In [ ]:
pl.fill_between(np.arange(1750, 2024), temp_all.quantile(0.05, dim="config"), temp_all.quantile(0.95, dim="config"), color='orange', alpha=0.2, lw=1)
pl.plot(np.arange(1750, 2024), temp_all.median(dim="config"), color='orange', label='All forcers');

pl.fill_between(np.arange(1750, 2024), temp_ghgs.quantile(0.05, dim="config"), temp_ghgs.quantile(0.95, dim="config"), color='0.5', alpha=0.2, lw=1)
pl.plot(np.arange(1750, 2024), temp_ghgs.median(dim="config"), color='0.5', label='Greenhouse gases');

pl.fill_between(np.arange(1750, 2024), temp_aerosols.quantile(0.05, dim="config"), temp_aerosols.quantile(0.95, dim="config"), color='blue', alpha=0.2, lw=1)
pl.plot(np.arange(1750, 2024), temp_aerosols.median(dim="config"), color='blue', label='Aerosols');

#pl.plot(np.arange(1750, 2024), temp_other.median(dim="config"), color='green')

pl.fill_between(np.arange(1750, 2024), temp_natural.quantile(0.05, dim="config"), temp_natural.quantile(0.95, dim="config"), color='green', alpha=0.2, lw=1)
pl.plot(np.arange(1750, 2024), temp_natural.median(dim="config"), color='green', label='Natural');
pl.plot(df_obs.gmst, color='k', label='4-dataset mean obs. (Forster et al. 2023)')

pl.ylabel('°C relative to 1850-1900')
pl.legend();
pl.grid()

pl.xlim(1850, 2023)
pl.ylim(-1.5, 2.5)

os.makedirs('../plots/', exist_ok=True)
pl.tight_layout()
pl.savefig('../plots/attributed_warming_rel1850-1900.png')

In [ ]:
temp_aerosols.median(dim="config")

In [ ]:
df_out = pd.DataFrame(
    np.array(
        [
            temp_ghgs.quantile(0.05, dim="config").data,
            temp_ghgs.median(dim="config").data,
            temp_ghgs.quantile(0.95, dim="config").data,
            temp_aerosols.quantile(0.05, dim="config").data,
            temp_aerosols.median(dim="config").data,
            temp_aerosols.quantile(0.95, dim="config").data,
            temp_other.quantile(0.05, dim="config").data,
            temp_other.median(dim="config").data,
            temp_other.quantile(0.95, dim="config").data,
            temp_natural.quantile(0.05, dim="config").data,
            temp_natural.median(dim="config").data,
            temp_natural.quantile(0.95, dim="config").data,
            temp_all.quantile(0.05, dim="config").data,
            temp_all.median(dim="config").data,
            temp_all.quantile(0.95, dim="config").data,
        ]
    ).T,
    index=np.arange(1750, 2024),
    columns=['ghg_05', 'ghg_50', 'ghg_95', 'aerosol_05', 'aerosol_50', 'aerosol_95', 'other_05', 'other_50', 'other_95', 'natural_05', 'natural_50', 'natural_95', 'all_05', 'all_50', 'all_95']
)

In [ ]:
os.makedirs('../output', exist_ok=True)
df_out.to_csv('../output/attributed_warming.csv')